* 1. Load the libries and Data For the project

In [85]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC,SVR
from sklearn.metrics import r2_score
import xgboost as xgb

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV


# Metrics


# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

store = pd.read_csv('./input/store.csv')
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

print store.head()

   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c                620.0                        9.0   
4      5         a          a              29910.0                        4.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0              NaN              NaN   
1                    2007.0       1             13.0           2010.0   
2                    2006.0       1             14.0           2011.0   
3                    2009.0       0              NaN              NaN   
4                    2015.0       0              NaN              NaN   

     PromoInterval  
0              NaN  
1  Jan,Apr,Jul,Oct  
2  Jan,Apr,

In [86]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()
    
def normalize(x):

    xmin = x.min()
    xmax = x.max()
    
    x = (x-xmin)/(xmax-xmin)

    return x

def rmspe_score(y_true,y_pred):
    return 'RMSPE',np.sqrt(np.mean((1-y_pred/y_true)**2))

def minmaxscaler(df):
    return 

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )


In [87]:
def build_features(features):
    features = pd.get_dummies(features,columns=['DayOfWeek','StoreType','Assortment','PromoInterval'])
    features['StateHoliday'].replace('a',1,inplace=True)
    features['StateHoliday'].replace('b',2,inplace=True)
    features['StateHoliday'].replace('c',3,inplace=True)
    features['StateHoliday'] =  pd.to_numeric(features['StateHoliday'], downcast='integer')
    features['StateHoliday'] = normalize(features["StateHoliday"])
    features['Date'] = pd.to_datetime(features['Date'])
    features['CompetitionOpenSinceMonth'] = pd.to_numeric(features['CompetitionOpenSinceMonth'], downcast='integer')
    features['CompetitionOpenSinceYear'] = pd.to_numeric(features['CompetitionOpenSinceYear'], downcast='integer')
    features['Month'] = features['Date'].map(lambda x: x.month)
    features['Year'] = features['Date'].map(lambda x: x.year)
    features['Week'] = features['Date'].map(lambda x: x.strftime("%V"))
    features['Week']=pd.to_numeric(features['Week'], downcast='integer')
    features["CompetitionMonthDuration"] = (12-features['CompetitionOpenSinceMonth']+features['Month'])+(features['Year']-features['CompetitionOpenSinceYear'])*12
    features["Promo2WeekDuration"] = (53-features['Promo2SinceWeek']+features['Week'])+(features['Year']-features['Promo2SinceYear'])*53
    features["CompetitionMonthDuration"] = normalize(features["CompetitionMonthDuration"])
    features["Promo2WeekDuration"] = normalize(features["Promo2WeekDuration"])
    features = features.drop(['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear','Month','Year','Week','Date','Store'],axis=1)
    if 'Customers' in features.columns:
        features = features.drop(['Customers'],axis=1)
    features.fillna(0,inplace=True)
    
    return features

In [88]:
store["CompetitionDistance"] = normalize(store["CompetitionDistance"])

org_train_data = pd.merge(train,store,on='Store')
org_test_data = pd.merge(test,store,on='Store')

sales_train_raw = org_train_data[org_train_data['Sales']!=0]['Sales']
sales_train = np.log1p(sales_train_raw)

features_train_raw = org_train_data[org_train_data['Sales']!=0].drop('Sales',axis=1)

features_train = build_features(features_train_raw)

#print features.head()

# 将'features'和'val'数据切分成训练集和测试集
X_train, X_val, y_train, y_val = train_test_split(features_train, sales_train, test_size = 0.1, random_state = 0)

xgdmat=xgb.DMatrix(X_train,y_train)
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.2,
          "max_depth": 12,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1
          }
num_trees = 200
final_gb=xgb.train(params,xgdmat,num_trees,feval=rmspe_score)
tesdmat=xgb.DMatrix(X_val)
y_pred=final_gb.predict(tesdmat)

#print r2_score(y_val,y_pred)
print rmspe_score(y_val,y_pred)

reg = LinearRegression()
#reg.fit(X_train,y_train)
#y_pred=reg.predict(X_val)

#print rmspe_score(y_val,y_pred)
#print reg.score(X_val,y_val)

('RMSPE', 0.01699552219123154)


In [89]:
test = build_features(org_test_data)
test_train = test.drop(['Id'],axis=1)
test_result = final_gb.predict(xgb.DMatrix(test_train))
submission = pd.DataFrame({"Id": test['Id'], "Sales": np.expm1(test_result)})
submission.to_csv("my_submission.csv", index=False)

In [90]:
# XGB feature importances
# Based on https://www.kaggle.com/mmueller/liberty-mutual-group-property-inspection-prediction/xgb-feature-importance-python/code

create_feature_map(features)
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

featp = df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
fig_featp = featp.get_figure()
fig_featp.savefig('feature_importance_xgb.png', bbox_inches='tight', pad_inches=1)

Exception AttributeError: "'DMatrix' object has no attribute 'handle'" in <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x10a2af190>> ignored


NameError: name 'gbm' is not defined